# Your First Binary Simulations with POSYDON 🌠

**Tutorial goal:**

In this tutorial, we will run a small population of 10 binaries locally, and explore how to manipulate the output data from your population.

**New concepts:**

- Population ini file

If you haven't done so yet, export the path POSYDON environment variables.
Set these parameters in your `.bash_profile` or `.zshrc` if you use POSYDON regularly.

In [1]:
%env PATH_TO_POSYDON=/YOUR/POSYDON/PATH/
%env PATH_TO_POSYDON_DATA=/YOUR/POSYDON_DATA/PATH/

env: PATH_TO_POSYDON=/Users/simone/Google Drive/github/POSYDON-public/
env: PATH_TO_POSYDON_DATA=/Volumes/T7/


## Creating the Initialisation File

----
To run population synthesis with POSYDON, a `population_params.ini` file is required.
This file described how the stellar population is created and what prescriptions and parameters are implemented in specific steps.

POSYDON comes with a default `population_params_default.ini` file found at `PATH_TO_POSYDON/posydon/popsyn` or have a look [here](population_params.ini).

The file is split in three main parts. You can find more details about their properties by clicking on their links.
1. **[SimulationProperties]():**
   - these describe the properties and parameters of different steps in the evolution of a binary systems.
2. **[BinaryPopulation]():** 
   - parameters of the initial sampling of the binary population, such as initial mass function, period distribution, and metallicity.
   - Also contains parameters on how the population is ran, such as how many binaries are kept in memory.
3. **[SavingOutput]()**
   - Describes the data from the binary and each individual star to the output files.

We will copy the default population run parameter file to the current folder.

In [1]:
import os
import shutil
from posydon.config import PATH_TO_POSYDON

path_to_params = os.path.join(PATH_TO_POSYDON, "posydon/popsyn/population_params_default.ini")
shutil.copyfile(path_to_params, './population_params.ini')

'./population_params.ini'

# Creating and running a binary population

The copied `population_params.ini` contains the parameters to run 10 binaries at a metallicity of $Z=10^{-4}$.

If you open the file and scroll down to the **BinaryPopulation** section, you will see how they're defined:

```
metallicity = [0.0001] # [2., 1., 0.45, 0.2, 0.1, 0.01, 0.001, 0.0001]
# In units of solar metallicity
...
number_of_binaries = 10
# int
```

If you like to run a small population in a notebook, you can use the `PopulationRunner` to do this. If you want to run a specific binary instead, have a look at the [Binary Tutorial]().

The `PopulationRunner` class takes the `.ini` file and sets-up a population run.

This will create [BinaryPopulations]() for each metallicity defined in the `.ini` file.
In this case, we can check and see that a single BinaryPopulation is created and contains 10 binaries.

In [2]:
from posydon.popsyn.synthetic_population import PopulationRunner
poprun = PopulationRunner('./population_params.ini')

In [3]:
print('Number of binary populations:',len(poprun.binary_populations))
print('Metallicity:', poprun.binary_populations[0].metallicity)
print('Number of binaries:', poprun.binary_populations[0].number_of_binaries)  

Number of binary populations: 1
Metallicity: 0.0001
Number of binaries: 10


After setting up the population run, we can evolve it. This should take about 20 seconds, but depends on your machine.

In [4]:
poprun.evolve()

# Inspecting the population: Population class

When you ran the population, you might have seen that a temporary folder with the name `1e-04_Zsun_batches` was created while the binaries were evolved.
This is a temporary folder in which populations are temporarly saved.
After the binary evolution has finished, the binaries in the folder are moved to a single file named `1e-04_Zsun_popululation.h5`. This is done automatically, when you run a population using the `PopulationRunner` class.

The created file contains 3 main components:

1. **history:** the evolution of an individual binary in a pandas DataFrame
2. **oneline:** a single line to describe the initial and final conditions and some one-of parameters, such as the metallicity.
3. **mass_per_met:** some metadata on the population, such as the total simulated mass, the actual underlying mass of the population, and the number of binaries in the file.

The `Population` provides an interface with these components in the file, such that you're able to share the populations runs and can work with large population that do not fit in memory.


<div class="alert alert-block alert-warning"><b>Older Population Files</b> 

If you're using older population files, you can make them compatible with the `Population` class by calling `Population(pop_file, metallicity,ini_file)`, where the `metallicity` is in solar units. You will only need to do this once; afterwards you can initialise the class like normal.<div>

In [6]:
from posydon.popsyn.synthetic_population import Population

In [7]:
pop = Population('1e-04_Zsun_population.h5')

The `pop.mas_per_met` shows some basic information about the population you've just created.


1. The index is the metallicity of your population in solar units.
2. **simulated mass** is the total ZAMS mass that has been evolved in the population.
3. **underlying mass** is the actual mass of the population if one integrates the IMF and period distribution fully.
4. **number_of_systems** shows the 10 systems in the file.

In [6]:
pop.mass_per_met

,simulated_mass,underlying_mass,number_of_systems
0.0001,283.329774,1593.39743,10


There are some additional metadata properties available, such as:
- `metallicities`: the metallicity in absolute metallicity
- `solar_metallicities`: the metallicities in the file in solar metallicity
- `number_of_systems`: the total number of systems in the Population file
- `indices`: the indices of the binaries in the file
- `columns`: the columns available in the `history` and `oneline` dataframes

In [37]:
# you can also access the total number of systems in the file with
print(pop.number_of_systems)

10


## Population.history

`pop.history` loads in the full history of all the binaries into memory.
You can access individual or a selections of the population using several methods:


1. pop.history[5]
2. pop.history[[0,4]]
3. pop.history['time]
4. pop.history.select()

The select function is the most powerfull way to access the binaries, because it allows you to perform selections based on the columns available in the history dataframe.

For example, below we can select on `state == 'RLO1'`, which gives us all the rows with RLO1 occuring.
At the same time, we can request only specific columns, 


In [11]:
# select only binary_index 5
pop.history[5]

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
5,detached,ZAMS,0.000000e+00,194.173343,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,14.090024,...,NaN,NaN,NaN,NaN,0.750999,0.24900,NaN,NaN,NaN,NaN
5,RLO1,oCE1,1.603818e+07,85.883938,0.0,-1.914671,step_HMS_HMS,0.008191,H-rich_Central_He_depleted,8.149206,...,0.0,0.0,0.0,0.0,0.636049,0.36395,0.743545,0.256452,0.989063,11.473863
5,merged,oMerging1,1.603818e+07,85.883938,0.0,-1.914671,step_CE,0.000033,H-rich_Central_He_depleted,8.149206,...,0.0,0.0,0.0,0.0,0.636049,0.36395,0.743545,0.256452,0.989063,11.473863
5,merged,CC1,1.604647e+07,NaN,NaN,NaN,step_merged,0.114275,H-rich_Central_C_depletion,14.026236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,merged,NaN,1.604647e+07,NaN,NaN,NaN,step_SN,0.000471,NS,1.302396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,merged,END,1.604647e+07,NaN,NaN,NaN,step_end,0.000017,NS,1.302396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# select binary 0 and 4
pop.history[[0,4]]

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
0,detached,ZAMS,0.000000e+00,2725.036696,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,17.710903,...,NaN,NaN,NaN,NaN,7.509986e-01,2.490000e-01,NaN,NaN,NaN,NaN
0,detached,CC1,1.191980e+07,2508.127760,0.0,-11.264136,step_HMS_HMS,0.666948,H-rich_Central_C_depletion,17.610367,...,0.000000,0.0,0.000000,0.000000,3.508430e-01,6.491556e-01,0.750688,0.249285,0.000300,0.001112
0,disrupted,NaN,1.191980e+07,NaN,NaN,NaN,step_SN,0.000411,NS,1.440761,...,0.000000,0.0,0.000000,0.000000,3.508430e-01,6.491556e-01,0.750688,0.249285,0.000300,0.001112
0,disrupted,CC2,1.715168e+07,NaN,NaN,NaN,step_disrupted,2.049328,NS,1.440761,...,5.217357,NaN,3.992467,0.076565,4.705053e-36,5.618140e-13,0.736146,0.263841,0.000002,0.001063
0,disrupted,NaN,1.715168e+07,NaN,NaN,NaN,step_SN,0.000521,NS,1.440761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
0,disrupted,END,1.715168e+07,NaN,NaN,NaN,step_end,0.000021,NS,1.440761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
4,detached,ZAMS,0.000000e+00,1.293195,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,8.140102,...,NaN,NaN,NaN,NaN,7.509986e-01,2.490000e-01,NaN,NaN,NaN,NaN
4,contact,oCE1,3.452328e+07,0.894911,0.0,-2.873830,step_HMS_HMS,0.008777,H-rich_Core_H_burning,5.720771,...,0.000000,0.0,0.000000,0.000000,4.798080e-01,5.201906e-01,0.748799,0.251183,0.758031,7.049825
4,merged,oMerging1,3.452328e+07,0.894911,0.0,-2.873830,step_CE,0.000020,H-rich_Core_H_burning,5.720771,...,0.000000,0.0,0.000000,0.000000,4.798080e-01,5.201906e-01,0.748799,0.251183,0.758031,7.049825


In [19]:
pop.history['time']

,time
binary_index,
0,0.000000e+00
0,1.191980e+07
0,1.191980e+07
0,1.715168e+07
0,1.715168e+07
...,...
9,9.452675e+06
9,9.716214e+06
9,9.716214e+06


You can also check what columns are available in the history file:

In [38]:
pop.history.columns

Index(['state', 'event', 'time', 'orbital_period', 'eccentricity',
       'lg_mtransfer_rate', 'step_names', 'step_times', 'S1_state', 'S1_mass',
       'S1_log_R', 'S1_log_L', 'S1_lg_mdot', 'S1_he_core_mass',
       'S1_he_core_radius', 'S1_co_core_mass', 'S1_co_core_radius',
       'S1_center_h1', 'S1_center_he4', 'S1_surface_h1', 'S1_surface_he4',
       'S1_surf_avg_omega_div_omega_crit', 'S1_spin', 'S2_state', 'S2_mass',
       'S2_log_R', 'S2_log_L', 'S2_lg_mdot', 'S2_he_core_mass',
       'S2_he_core_radius', 'S2_co_core_mass', 'S2_co_core_radius',
       'S2_center_h1', 'S2_center_he4', 'S2_surface_h1', 'S2_surface_he4',
       'S2_surf_avg_omega_div_omega_crit', 'S2_spin'],
      dtype='object')

In [21]:
# using the select function
pop.history.select(where='index == 9')
# selecting all RLO1 states and only time and state columns
pop.history.select(where='state == RLO1', columns=['time', 'state'])
# selecting rows 10 to 16
pop.history.select(start=10, stop=16)


,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
1,merged,NaN,4.074141e+06,NaN,NaN,NaN,step_SN,0.027684,BH,34.957749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,merged,END,4.074141e+06,NaN,NaN,NaN,step_end,0.000019,BH,34.957749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,detached,ZAMS,0.000000e+00,444.366230,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,18.603707,...,NaN,NaN,NaN,NaN,7.509986e-01,2.490000e-01,NaN,NaN,NaN,NaN
2,RLO1,CC1,1.124996e+07,893.254535,0.0,-5.795808,step_HMS_HMS,0.143167,H-rich_Central_C_depletion,9.380271,...,0.00000,0.0,0.000000,0.000000,2.384658e-01,7.615328e-01,0.484047,0.515949,0.894986,11.451812
2,disrupted,NaN,1.124996e+07,NaN,NaN,NaN,step_SN,0.000375,NS,1.439520,...,0.00000,0.0,0.000000,0.000000,2.384658e-01,7.615328e-01,0.484047,0.515949,0.894986,11.451812
2,disrupted,CC2,1.401204e+07,NaN,NaN,NaN,step_disrupted,0.246076,NS,1.439520,...,6.55894,NaN,5.243425,0.088777,9.312553e-32,1.196424e-12,0.736902,0.263085,0.022222,10.363551


You might have notices while using the above functions that not all the binaries will have the same length in the history.
You can access these with `pop.history_lengths`. This information is also stored in the population file.

In [24]:
pop.history_lengths

,index
index,
0,6
1,6
2,6
3,6
4,6
5,6
6,6
7,6
8,6


## Population.oneline

`Population.oneline` provides a similar interface to accessing the DataFrame in the population file as `Population.history`, with similar functionality being available.

In [36]:
pop.oneline[5]
pop.oneline[[0,4]]
pop.oneline.select(where='index == 9')
pop.oneline.select(where='S1_mass_i > 20', columns=['state_i','S1_state_i', 'S1_mass_i', 'S2_state_i','S2_mass_i'])

,state_i,S1_state_i,S1_mass_i,S2_state_i,S2_mass_i
binary_index,,,,,
1,detached,H-rich_Core_H_burning,68.590575,H-rich_Core_H_burning,4.450514
9,detached,H-rich_Core_H_burning,25.002056,H-rich_Core_H_burning,21.094360


In [39]:
pop.oneline.columns

Index(['state_i', 'event_i', 'time_i', 'orbital_period_i', 'eccentricity_i',
       'lg_mtransfer_rate_i', 'step_names_i', 'step_times_i', 'state_f',
       'event_f',
       ...
       'interp_class_CO_HMS_RLO', 'interp_class_CO_HeMS',
       'interp_class_CO_HeMS_RLO', 'mt_history_HMS_HMS',
       'mt_history_CO_HMS_RLO', 'mt_history_CO_HeMS', 'mt_history_CO_HeMS_RLO',
       'FAILED', 'WARNING', 'metallicity'],
      dtype='object', length=101)

## Population.formation_channels

You might be interested in figuring out what sort of formation pathways/channels a binary has followed through its evolution.

This is not a standard output of the population synthesis, but you can include it into the population file by calculating it. 
If you would like more detail on the initial mass transfer, you can set `mt_history=True`.

This will write the formation channels to the Population file, which can be accessed by `Population.formation_channels`.


In [46]:
pop.calculate_formation_channels(mt_history=True)

Formation channels already exist in the parsed population file!
Channels will be overwriten
62 62


In [47]:
# the formation channels are loaded in with pop.formation_channels
pop.formation_channels

,channel_debug,channel
0,ZAMS_CC1_CC2_END,ZAMS_CC1_CC2_END
1,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
2,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
3,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
4,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
5,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
6,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
7,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
8,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
9,ZAMS_oRLO1_CC1_oRLO2_CC2_maxtime_END,ZAMS_oRLO1_CC1_oRLO2_CC2_maxtime_END


Next time you open this population file, the `formation_channels` will be available without having to be recalculated.

In [48]:
pop = Population('1e-04_Zsun_population.h5')
pop.formation_channels

,channel_debug,channel
0,ZAMS_CC1_CC2_END,ZAMS_CC1_CC2_END
1,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
2,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
3,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
4,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
5,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
6,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
7,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
8,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
9,ZAMS_oRLO1_CC1_oRLO2_CC2_maxtime_END,ZAMS_oRLO1_CC1_oRLO2_CC2_maxtime_END


## Selecting a sub-population

You might just want a small sub-selection of the full population, especially if you're working with large population and multi-metallicity runs.

The `Population.export_selection()` function will export just the indices of the binaries you're interested in into a new file.
The simulated and underlying mass will remain the same, since they are dependent on the population run.

If we select just 2 binaries and export them, we create a new population of just the binaries you're interested in.
In the [BBH analysis]() and [GRB analysis]() tutorials, we show how to perform a selection with multiple criteria and metallicities.



In [8]:
indices = [0,9]
pop.export_selection(indices, 'selected.h5')

/Users/max/Documents/POSYDON/posydon/popsyn/synthetic_population.py:628: UserWarning: No formation channels in the population file!
  warnings.warn('No formation channels in the population file!')


In [10]:
selected = Population('selected.h5')
selected.mass_per_met

,simulated_mass,underlying_mass,number_of_systems
0.0001,250.829612,1410.622167,2


<br/><br/>

Feel free to explore the small binary population you've just created!

If you want to learn more about population synthesis and how to build more complex models it is advised to continue with the remaining tutorials and consult the POSYDON documentation.

### Local MPI runs

To speed up population synthesis runs, you can run on a computing cluster, as described in [HPC Facilities](pop_syn), or you can distribute the population synthesis across multiple cores on your local machine using MPI.

To enable local MPI runs, go into the `population_params.ini`  and change `use_MPI` to `True`.

It's important to note that you cannot run have this option enabled for cluster runs!

We create a binary population simulation script to run the population:

In [ ]:
%%writefile script.py
from posydon.popsyn.synthetic_population import PopulationRunner

if __name__ == "__main__":
    synth_pop = PopulationRunner("./population_params.ini")
    synth_pop.evolve()

This script can be initiated using a local where `NR_processors` is the number of processors you would like to us.

In [ ]:
mpiexec -n ${NR_processors} python script.py

This will create a folder for each metallicity in the population and store output of the parallel runs in it.

You will have to concatenate these runs manually into a single population file per metallicity, which can be achieved using the following code:

In [ ]:
from posydon.popsyn.synthetic_population import PopulationRunner

synth_pop = PopulationRunner("./population_params.ini")
for pop in synth_pop.binary_populations:
    synth_pop.merge_parallel_runs(pop)